- notebook/requirements.txt: EDA를 위한 종속성 관리
- autoEDA: llama2로 특수 문자 여부를 판단한 내용

In [1]:
import pandas as pd
import pyarrow as pa
import numpy as np
import string
from collections import Counter
from langchain_ollama.llms import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
file_path = '../data/train_dataset/train/dataset.arrow'
with pa.memory_map(file_path, 'r') as source:
    table = pa.ipc.open_stream(source).read_all()
df: pd.DataFrame = table.to_pandas()
df.head()

\* 로컬 Ollama 서버로 llama2 활용하는 방법 (macOS 기준)
1. Ollama 설치
    ```
    brew install ollama
    ```
2. Ollama 서버 실행
    ```
    ollama run llama2
    ```
3. Ollama 서버 중단
    ```
    pkill ollama
    ```

In [3]:
llm = OllamaLLM(model="llama2")

In [4]:
summary_prompt_template = """
다음 문서를 분석하고 주어진 질문에 답변할 수 있도록 필요한 정보를 요약해 주세요. 문서의 논리 구조와 질문의 연관성을 중점적으로 파악하고, 관련 없는 세부 사항은 제외하세요.

### 문서:
{context}

### 질문:
{question}

문서를 **한국어로 요약**하고, 질문에 대한 답변에 필요한 정보만 포함해 주세요.
"""
summary_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=summary_prompt_template
)

In [5]:
answer_prompt_template = """
요약된 문서를 바탕으로 주어진 질문에 **오직 한국어로** 짧고 간결하게 대답해 주세요. 답변은 **한 단어나 짧은 구절**로 이루어져야 합니다.

### 요약된 문서:
{summary}

### 질문:
{question}

**한국어로** 가장 중요한 한 단어나 구절로 대답해 주세요. **영어로 대답하지 마세요.**
"""
answer_prompt = PromptTemplate(
    input_variables=["summary", "question"],
    template=answer_prompt_template
)

In [6]:
final_prompt_template = """
생성된 답변을 검토하여 다음 기준을 충족하는지 확인하세요:
1. 답변은 **한 단어나 짧은 구절**로, **한국어로만** 이루어져야 합니다.
2. 답변은 가능한 한 짧고 명확하게 작성되어야 합니다.
3. 질문에 직접적으로 관련된 내용만 포함되어야 하며, 불필요한 정보는 제외해야 합니다.

### 생성된 답변:
{answer}

답변이 너무 길거나 줄일 수 있는 경우, **한국어로** 더 짧고 간결하게 수정하세요. 최종 답변은 짧고 구체적이며 질문에 정확히 대답할 수 있는 **한국어** 단어로 작성되어야 합니다.
"""
final_prompt = PromptTemplate(
    input_variables=["answer"],
    template=final_prompt_template
)

In [7]:
# 각 프롬프트를 체인으로 연결
summary_chain = (
    {
        "context": RunnablePassthrough(),
        "question": RunnablePassthrough(),
    }
    | summary_prompt
    | llm
    | JsonOutputParser()
)

answer_chain = (
    {
        "summary": RunnablePassthrough(),
        "question": RunnablePassthrough(),
    }
    | answer_prompt
    | llm
    | JsonOutputParser()
)

final_chain = (
    {
        "answer": RunnablePassthrough(),
    }
    | final_prompt
    | llm
    | JsonOutputParser()
)

In [8]:
def normalize_korean_answer(s):
    def remove_punctuation(text):
        exclude = set(string.punctuation + "·、，．？！＂＇〃《》「」『』〔〕“”‘’〈〉【】()[]{}")
        return "".join(ch for ch in text if ch not in exclude)

    def white_space_fix(text):
        return " ".join(text.split())

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punctuation(lower(s)))

def exact_match(prediction, ground_truth):
    return normalize_korean_answer(prediction) == normalize_korean_answer(ground_truth)

# F1 Score calculation
def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_korean_answer(prediction).split()
    ground_truth_tokens = normalize_korean_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())

    if num_same == 0:
        return 0.0

    precision = num_same / len(prediction_tokens)
    recall = num_same / len(ground_truth_tokens)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

In [ ]:
exact_matches = []
f1_scores = []

for idx in range(len(df)):
    context = df.iloc[idx]['context']
    question = df.iloc[idx]['question']
    answer = df.iloc[idx]['answers']['text'][0]  # 정답 추출

    # Step 1: 문서 요약
    summary_result = summary_chain.invoke({
        "context": context,
        "question": question
    })

    # Step 2: 요약을 바탕으로 답변 생성
    answer_result = answer_chain.invoke({
        "summary": summary_result['answer'],
        "question": question
    })

    # Step 3: 최종 답변 검토 및 출력
    final_answer = final_chain.invoke({
        "answer": answer_result['answer'],
    })

    predicted_answer = final_answer['answer']
    
    # 정확도 평가 (Exact Match와 F1 Score)
    em = exact_match(predicted_answer, answer)
    f1 = f1_score(predicted_answer, answer)

    # 메트릭 저장
    exact_matches.append(em)
    f1_scores.append(f1)
    
    # 출력
    print(f"{idx}.")
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"True Answer: {answer}")
    print(f"Exact Match: {em}, F1 score: {f1}")
    print("-" * 40)

# 10. 전체 평균 메트릭 계산
avg_em = np.mean(exact_matches)
avg_f1 = np.mean(f1_scores)

print(f"Average Exact Match: {avg_em:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")